# Batch Shapefile → GeoTIFF Converter (with Geometry Repair & Size Limits)

This notebook **batch‑converts every `.shp` file beneath a chosen root directory into GeoTIFF rasters** while:

1. repairing invalid geometries to avoid artefacts (e.g., stray horizontal lines);
2. ensuring the output raster is **≤ 4096 × 4096** pixels (adjusting pixel size if necessary; default target ≈ `0.00001` degree);
3. saving each GeoTIFF inside a sibling `tif/` folder alongside its source shapefile;
4. writing a quick‑look *PNG* overlay (shapefile boundary + raster) for visual QA.

> **Dependencies**  
> - GeoPandas ≥ 0.14  
> - Shapely ≥ 2.0  
> - Rasterio ≥ 1.3  
> - Matplotlib  
> 
> Install (conda):  
> ```bash
> conda create -n gis python=3.11 geopandas rasterio matplotlib -c conda-forge
> conda activate gis
> ```


In [ ]:
import warnings, math, pathlib, sys
from pathlib import Path

import geopandas as gpd
from shapely import make_valid
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import matplotlib.pyplot as plt


## Configure I/O Paths

In [ ]:
# 🔧 CHANGE THESE PATHS TO YOUR OWN
input_root  = Path(r'D:/gis/shp_root')   # root folder that **contains** .shp files (scanned recursively)
output_root = Path(r'D:/gis/tif_root')   # where converted GeoTIFFs & previews will be written

output_root.mkdir(parents=True, exist_ok=True)
print('Input root:', input_root.resolve())
print('Output root:', output_root.resolve())


## Constants

In [ ]:
PIXEL_LIMIT   = 4096      # maximum rows or columns
PIXEL_SIZE_MAX = 1e-5     # desired (finest) pixel size in coordinate units (deg)


## Helper Functions

In [ ]:
def repair_geometries(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Attempt to fix invalid geometries."""
    def _fix(geom):
        if geom is None or geom.is_empty:
            return None
        if not geom.is_valid:
            try:
                return make_valid(geom)
            except Exception:
                return geom.buffer(0)
        return geom

    gdf['geometry'] = gdf.geometry.apply(_fix)
    gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notnull()]
    return gdf

def choose_pixel_size(bbox, finest=PIXEL_SIZE_MAX, limit=PIXEL_LIMIT):
    minx, miny, maxx, maxy = bbox
    dx, dy = maxx - minx, maxy - miny
    if dx / finest <= limit and dy / finest <= limit:
        return finest
    px = max(dx, dy) / limit
    if px > finest:
        warnings.warn(f'Pixel size relaxed to {px:.6f} to satisfy {limit}×{limit} cap → resolution loss.')
    return px


In [ ]:
def shp_to_tif(shp_path: Path, out_root: Path):
    name = shp_path.stem
    out_dir = (out_root / name / 'tif')
    out_dir.mkdir(parents=True, exist_ok=True)

    out_tif = out_dir / f'{name}.tif'
    out_png = out_dir / f'{name}_preview.png'

    gdf = gpd.read_file(shp_path)
    gdf = repair_geometries(gdf)         # 🚫 NO reprojection assumed (all files already share CRS)
    if gdf.empty:
        print(f'[SKIP] {shp_path.name} – empty after repair.')
        return

    minx, miny, maxx, maxy = gdf.total_bounds
    px = choose_pixel_size((minx, miny, maxx, maxy))
    width  = math.ceil((maxx - minx) / px)
    height = math.ceil((maxy - miny) / px)

    transform = from_origin(minx, maxy, px, px)

    shapes = ((geom, 1) for geom in gdf.geometry)
    raster = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0,
        dtype='uint8'
    )

    meta = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': 1,
        'dtype': 'uint8',
        'transform': transform,
        'crs': gdf.crs,
        'nodata': 0
    }

    with rasterio.open(out_tif, 'w', **meta) as dst:
        dst.write(raster, 1)

    # Quick visual overlay
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(raster, extent=[minx, maxx, miny, maxy], origin='upper', alpha=0.6, cmap='gray')
    gdf.boundary.plot(ax=ax, edgecolor='red', linewidth=0.4)
    ax.set_title(f'{name}: shp vs tif')
    ax.set_axis_off()
    plt.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)

    print(f'[OK] {shp_path.name} → {out_tif.name}  ({width}×{height}, px={px:.6f})')


## Batch Processing

In [ ]:
shp_files = list(input_root.rglob('*.shp'))
print(f'Found {len(shp_files)} shapefile(s).')

for shp in shp_files:
    try:
        shp_to_tif(shp, output_root)
    except Exception as exc:
        print(f'[ERR] {shp.name}: {exc}')
